In [1]:
import glob
import os
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime

def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)  # メル周波数ケプストラム係数（Mel-Frequency Cepstrum Coefficients）
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)  # クロマベクトル(Chroma)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)  # メル周波数ケプストラムの平均値
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)  # スペクトル-コントラクト(Spectral Contrast)の平均
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)  #  和音に関する特徴量(tonal centroid features)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir, sub_dirs, file_ext='*.wav', verbose=0):
    features, labels = np.empty((0,193)), np.empty(0)    # 193次元の特徴量配列
    for label, sub_dir in enumerate(sub_dirs):
        files = glob.glob(os.path.join(parent_dir, sub_dir, file_ext))
        for fn in files:
            if (verbose):
                print('loading: ' + fn)
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])   # hstack: 配列を横方向に結合
            features = np.vstack([features,ext_features])
            paths = fn.split('/')
            labels = np.append(labels, paths[len(paths)-1].split('-')[1])
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):  # ラベルをスカラー形式からベクトル形式に変換
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [10]:
def load_urbansound8k(parent_dir, sub_dirs, verbose=0):
    file_ext = '*.wav'
    features, labels = parse_audio_files(parent_dir, sub_dirs, file_ext, verbose)
    labels = one_hot_encode(labels)
    return features, labels

def load_urbansound8k_train_and_test(parent_dir = 'UrbanSound8K/audio/',
                                     tr_sub_dirs = ['fold1', 'fold2'], 
                                     ts_sub_dirs = ['fold3'],
                                     verbose = 0):
    tr_features, tr_labels = load_urbansound8k(parent_dir, tr_sub_dirs, verbose)
    ts_features, ts_labels = load_urbansound8k(parent_dir, ts_sub_dirs, verbose)
    return tr_features, tr_labels, ts_features, ts_labels

In [11]:
def save_to_npy_files(tr_features, tr_labels, ts_features, ts_labels, prefix=''):
    np.save(prefix + 'tr_features.npy', tr_features)
    np.save(prefix + 'tr_labels.npy', tr_labels)
    np.save(prefix + 'ts_features.npy', ts_features)
    np.save(prefix + 'ts_labels.npy', ts_labels)

In [12]:
def load_from_npy_files(prefix = ''):
    tr_features = np.load(prefix + 'tr_features.npy')
    tr_labels = np.load(prefix + 'tr_labels.npy')
    ts_features = np.load(prefix + 'ts_features.npy')
    ts_labels = np.load(prefix + 'ts_labels.npy')
    return tr_features, tr_labels, ts_features, ts_labels

In [9]:
if __name__ == '__main__':
    parent_dir = 'UrbanSound8K/audio/'
    tr_subdirs = ['fold1','fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7']
    ts_subdirs =['fold8', 'fold9', 'fold10']
    
    print('loading urbansound8k dataset.')    
    %time tr_features, tr_labels, ts_features, ts_labels = load_urbansound8k_train_and_test(parent_dir,  tr_subdirs, ts_subdirs, 1)
    save_to_npy_files(tr_features, tr_labels, ts_features, ts_labels, prefix='all_')


loading urbansound8k dataset.
loading: UrbanSound8K/audio/fold8/103076-3-0-0.wav
loading: UrbanSound8K/audio/fold8/103076-3-1-0.wav
loading: UrbanSound8K/audio/fold8/103076-3-2-0.wav
loading: UrbanSound8K/audio/fold8/106905-5-0-0.wav
loading: UrbanSound8K/audio/fold8/106905-5-0-1.wav
loading: UrbanSound8K/audio/fold8/106905-5-0-2.wav
loading: UrbanSound8K/audio/fold8/107090-1-0-0.wav
loading: UrbanSound8K/audio/fold8/107090-1-1-0.wav
loading: UrbanSound8K/audio/fold8/107190-3-0-0.wav
loading: UrbanSound8K/audio/fold8/110134-3-0-0.wav
loading: UrbanSound8K/audio/fold8/110688-3-0-11.wav
loading: UrbanSound8K/audio/fold8/110688-3-0-17.wav
loading: UrbanSound8K/audio/fold8/110688-3-0-2.wav
loading: UrbanSound8K/audio/fold8/110688-3-0-32.wav
loading: UrbanSound8K/audio/fold8/113202-5-0-0.wav
loading: UrbanSound8K/audio/fold8/113202-5-0-1.wav
loading: UrbanSound8K/audio/fold8/113202-5-0-10.wav
loading: UrbanSound8K/audio/fold8/113202-5-0-11.wav
loading: UrbanSound8K/audio/fold8/113202-5-0-12

loading: UrbanSound8K/audio/fold8/133473-8-0-1.wav
loading: UrbanSound8K/audio/fold8/133473-8-0-2.wav
loading: UrbanSound8K/audio/fold8/133473-8-0-3.wav
loading: UrbanSound8K/audio/fold8/133473-8-0-4.wav
loading: UrbanSound8K/audio/fold8/133473-8-0-5.wav
loading: UrbanSound8K/audio/fold8/138465-1-0-0.wav
loading: UrbanSound8K/audio/fold8/139000-4-0-0.wav
loading: UrbanSound8K/audio/fold8/139000-4-0-1.wav
loading: UrbanSound8K/audio/fold8/139000-4-0-2.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-0.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-1.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-19.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-20.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-21.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-24.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-30.wav
loading: UrbanSound8K/audio/fold8/142003-2-0-8.wav
loading: UrbanSound8K/audio/fold8/14386-9-0-11.wav
loading: UrbanSound8K/audio/fold8/14386-9-0-16.wav
loading: UrbanSound8K/audi

loading: UrbanSound8K/audio/fold8/167702-4-0-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-1-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-2-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-3-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-4-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-5-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-6-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-7-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-7-1.wav
loading: UrbanSound8K/audio/fold8/167702-4-8-0.wav
loading: UrbanSound8K/audio/fold8/167702-4-9-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-0-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-1-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-10-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-11-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-12-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-13-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-2-0.wav
loading: UrbanSound8K/audio/fold8/168037-4-3-0.wav
loading: UrbanSound8K/audio

loading: UrbanSound8K/audio/fold8/196057-2-0-0.wav
loading: UrbanSound8K/audio/fold8/196068-2-0-0.wav
loading: UrbanSound8K/audio/fold8/196068-2-0-1.wav
loading: UrbanSound8K/audio/fold8/196074-2-0-0.wav
loading: UrbanSound8K/audio/fold8/196561-3-0-16.wav
loading: UrbanSound8K/audio/fold8/196561-3-0-29.wav
loading: UrbanSound8K/audio/fold8/196561-3-0-44.wav
loading: UrbanSound8K/audio/fold8/196561-3-0-9.wav
loading: UrbanSound8K/audio/fold8/204526-2-0-121.wav
loading: UrbanSound8K/audio/fold8/204526-2-0-134.wav
loading: UrbanSound8K/audio/fold8/204526-2-0-153.wav
loading: UrbanSound8K/audio/fold8/204526-2-0-160.wav
loading: UrbanSound8K/audio/fold8/204526-2-0-166.wav
loading: UrbanSound8K/audio/fold8/204526-2-0-193.wav
loading: UrbanSound8K/audio/fold8/204526-2-0-71.wav
loading: UrbanSound8K/audio/fold8/205610-4-0-0.wav
loading: UrbanSound8K/audio/fold8/205610-4-0-1.wav
loading: UrbanSound8K/audio/fold8/205610-4-0-2.wav
loading: UrbanSound8K/audio/fold8/205610-4-0-3.wav
loading: UrbanS

loading: UrbanSound8K/audio/fold8/66324-9-0-42.wav
loading: UrbanSound8K/audio/fold8/66324-9-0-53.wav
loading: UrbanSound8K/audio/fold8/66324-9-0-54.wav
loading: UrbanSound8K/audio/fold8/66599-9-0-11.wav
loading: UrbanSound8K/audio/fold8/66599-9-0-17.wav
loading: UrbanSound8K/audio/fold8/66599-9-1-11.wav
loading: UrbanSound8K/audio/fold8/66599-9-1-21.wav
loading: UrbanSound8K/audio/fold8/66599-9-1-23.wav
loading: UrbanSound8K/audio/fold8/66599-9-1-4.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-0.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-1.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-2.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-3.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-4.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-5.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-6.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-7.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-8.wav
loading: UrbanSound8K/audio/fold8/68080-7-0-9.wav
loading: UrbanSound8K/audio/fold8/6984-1-0

...

loading: UrbanSound8K/audio/fold10/14470-2-0-65.wav
loading: UrbanSound8K/audio/fold10/14470-2-0-70.wav
loading: UrbanSound8K/audio/fold10/14470-2-0-72.wav
loading: UrbanSound8K/audio/fold10/146244-4-0-0.wav
loading: UrbanSound8K/audio/fold10/147491-9-0-0.wav
loading: UrbanSound8K/audio/fold10/147491-9-0-5.wav
loading: UrbanSound8K/audio/fold10/147491-9-2-10.wav
loading: UrbanSound8K/audio/fold10/147491-9-2-25.wav
loading: UrbanSound8K/audio/fold10/147491-9-2-32.wav
loading: UrbanSound8K/audio/fold10/147491-9-2-6.wav
loading: UrbanSound8K/audio/fold10/155241-9-0-50.wav
loading: UrbanSound8K/audio/fold10/155241-9-0-56.wav
loading: UrbanSound8K/audio/fold10/155241-9-0-69.wav
loading: UrbanSound8K/audio/fold10/155241-9-0-79.wav
loading: UrbanSound8K/audio/fold10/155241-9-0-8.wav
loading: UrbanSound8K/audio/fold10/155241-9-0-88.wav
loading: UrbanSound8K/audio/fold10/155262-2-0-101.wav
loading: UrbanSound8K/audio/fold10/155262-2-0-11.wav
loading: UrbanSound8K/audio/fold10/155262-2-0-15.wav


loading: UrbanSound8K/audio/fold10/164377-9-1-50.wav
loading: UrbanSound8K/audio/fold10/164377-9-1-92.wav
loading: UrbanSound8K/audio/fold10/164646-6-0-0.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-0.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-1.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-2.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-3.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-4.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-5.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-6.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-7.wav
loading: UrbanSound8K/audio/fold10/165166-8-0-8.wav
loading: UrbanSound8K/audio/fold10/165640-4-0-0.wav
loading: UrbanSound8K/audio/fold10/165640-4-0-1.wav
loading: UrbanSound8K/audio/fold10/165640-4-0-2.wav
loading: UrbanSound8K/audio/fold10/167464-0-0-0.wav
loading: UrbanSound8K/audio/fold10/167464-0-0-1.wav
loading: UrbanSound8K/audio/fold10/167464-0-0-10.wav
loading: UrbanSound8K/audio/fold10/167464-0-0-11.wav
loading:

loading: UrbanSound8K/audio/fold10/188813-7-2-0.wav
loading: UrbanSound8K/audio/fold10/188813-7-3-0.wav
loading: UrbanSound8K/audio/fold10/188813-7-4-0.wav
loading: UrbanSound8K/audio/fold10/188813-7-5-0.wav
loading: UrbanSound8K/audio/fold10/188813-7-6-0.wav
loading: UrbanSound8K/audio/fold10/188813-7-7-0.wav
loading: UrbanSound8K/audio/fold10/188813-7-8-0.wav
loading: UrbanSound8K/audio/fold10/188813-7-9-0.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-11.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-13.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-16.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-17.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-18.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-19.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-2.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-20.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-23.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-27.wav
loading: UrbanSound8K/audio/fold10/189982-0-0-28.wav
lo

loading: UrbanSound8K/audio/fold10/39847-5-0-10.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-11.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-12.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-2.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-3.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-4.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-5.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-6.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-7.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-8.wav
loading: UrbanSound8K/audio/fold10/39847-5-0-9.wav
loading: UrbanSound8K/audio/fold10/39847-5-1-0.wav
loading: UrbanSound8K/audio/fold10/41306-3-0-10.wav
loading: UrbanSound8K/audio/fold10/41306-3-0-13.wav
loading: UrbanSound8K/audio/fold10/41306-3-0-4.wav
loading: UrbanSound8K/audio/fold10/51024-3-0-16.wav
loading: UrbanSound8K/audio/fold10/51024-3-0-2.wav
loading: UrbanSound8K/audio/fold10/51024-3-0-23.wav
loading: UrbanSound8K/audio/fold10/51024-3-0-3.wav
loading: UrbanSound8K/au

loading: UrbanSound8K/audio/fold10/85574-3-0-0.wav
loading: UrbanSound8K/audio/fold10/85574-3-0-12.wav
loading: UrbanSound8K/audio/fold10/85574-3-0-2.wav
loading: UrbanSound8K/audio/fold10/85574-3-0-8.wav
loading: UrbanSound8K/audio/fold10/85665-3-0-0.wav
loading: UrbanSound8K/audio/fold10/88121-8-0-0.wav
loading: UrbanSound8K/audio/fold10/88121-8-0-1.wav
loading: UrbanSound8K/audio/fold10/88121-8-0-2.wav
loading: UrbanSound8K/audio/fold10/88121-8-0-3.wav
loading: UrbanSound8K/audio/fold10/88121-8-1-0.wav
loading: UrbanSound8K/audio/fold10/88121-8-2-0.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-0.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-1.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-10.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-11.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-12.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-13.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-14.wav
loading: UrbanSound8K/audio/fold10/93567-8-0-15.wav
loading: UrbanSound8K/au